In [7]:
import pandas as pd
import glob
import os
import csv

def load_split_csv(file_prefix, folder='data'):
    
    search_pattern = os.path.join(folder, f"{file_prefix}*")
    files = sorted(glob.glob(search_pattern))
    
    if not files:
        print(search_pattern)
        return pd.DataFrame()
    
    
    df_list = []
    for f in files:
        try:
            # quoting=csv.QUOTE_NONE 
           
            df = pd.read_csv(
                f, 
                engine='python', 
                quoting=csv.QUOTE_NONE, 
                on_bad_lines='skip', 
                encoding='utf-8'
            )
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {f}: {e}")
        

           
    
    if not df_list:
        return pd.DataFrame()
        
    return pd.concat(df_list, ignore_index=True)

df_movie = load_split_csv('movie_part_', folder='../data')
df_people = load_split_csv('people_temp_part_', folder='../data')
df_tmdb = load_split_csv('tmdb_full_part_', folder='.')

print(f"Movie rows: {len(df_movie)}")
print(f"People rows: {len(df_people)}")
print(f"TMDB rows: {len(df_tmdb)}")

Movie rows: 79088
People rows: 2981595
TMDB rows: 64715


In [8]:

df = pd.read_csv('../data/movie.csv')
df.shape


(218148, 9)

In [9]:
#recommandation par GENRES
#Préparer les GENRES pour la recommandation
df['genres'].head()


0                        ['Comedy']
1              ['Drama', 'Romance']
2      ['Drama', 'Comedy', 'Crime']
3    ['Drama', 'Comedy', 'Romance']
4               ['Crime', 'Comedy']
Name: genres, dtype: object

In [10]:
# transformer ce texte en vraie liste
#mettre tout en minuscule
#préparer une version “texte” pour calculer la similarité
import ast

df['genres_list'] = df['genres'].apply(ast.literal_eval)  #convertir le texte en liste
df['genres_list'].head()


0                    [Comedy]
1            [Drama, Romance]
2      [Drama, Comedy, Crime]
3    [Drama, Comedy, Romance]
4             [Crime, Comedy]
Name: genres_list, dtype: object

In [12]:
df['genres_text'] = df['genres_list'].apply(
    lambda x: ' '.join([g.lower() for g in x])
)

df[['title', 'genres_text']].head()


,title,genres_text
0,Blondie,comedy
1,Love at Twenty,drama romance
2,Ariel,drama comedy crime
3,Shadows in Paradise,drama comedy romance
4,Four Rooms,crime comedy


In [11]:
#préparer une version “texte” pour calculer la similarité
df['genres_text'] = df['genres_list'].apply(lambda x: ' '.join(x))
df[['title', 'genres_text']].head()


,title,genres_text
0,Blondie,Comedy
1,Love at Twenty,Drama Romance
2,Ariel,Drama Comedy Crime
3,Shadows in Paradise,Drama Comedy Romance
4,Four Rooms,Crime Comedy


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['genres_text'])

tfidf_matrix.shape


(218148, 21)

In [14]:
from sklearn.neighbors import NearestNeighbors #iKNN non supervise, importer le modèle de k plus proches voisins

knn = NearestNeighbors(
    n_neighbors=6,   # 1 = le film lui-même + 5 recommandations
    metric='euclidean'
)

knn.fit(tfidf_matrix)


,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",6
,"radius radius: float, default=1.0Range of parameter space to use by default for :meth:`radius_neighbors`queries.",1.0
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'euclidean'
,"p p: float (positive), default=2Parameter for the Minkowski metric fromsklearn.metrics.pairwise.pairwise_distances. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None


In [15]:
def recommend_by_genres(title, df=df, model=knn, n_reco=5):
    # verifier que le film existe
    if title not in df['title'].values:
        return "Film non trouvé"

    # recupiration  l'index du film
    idx = df[df['title'] == title].index[0]

    #hercher les voisins avec KNN
    distances, indices = model.kneighbors(
        tfidf_matrix[idx],
        n_neighbors=n_reco + 1
    )

    # enlever le film lui-même
    similar_indices = indices[0][1:]

    # retourner les films recommandés
    return df.loc[similar_indices, ['title', 'genres_text', 'rating']]


In [16]:
#Tester la fonction
recommend_by_genres("Blondie")


,title,genres_text,rating
218103,Buying a Cow,comedy,4.5
218109,The Barometer of Fidelity,comedy,5.0
89916,Scrambled Eggs,comedy,3.5
89979,Mimics Action 500,comedy,4.3
89934,Girl Hate,comedy,4.2
